<a href="https://colab.research.google.com/github/j-chenn/COMP551_Project_1/blob/main/COMP551_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Acquire, pre-process and analyze the data
## Acquiring both datasets:
Dataset 1: [Search Trends](https://github.com/google-research/open-covid-19-data/blob/master/data/exports/search_trends_symptoms_dataset/README.mdhttps://)

Dataset 2: [COVID hospitalization cases](https://github.com/google-research/open-covid-19-data)

In [5]:
# Imports

import numpy as np
import pandas as pd


In [ ]:
st_url = "https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_weekly_symptoms_dataset.csv"
hosp_url = "https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv"

# Load into pandas dataframes
st_df = pd.read_csv(st_url)
hosp_df = pd.read_csv(hosp_url)

In [36]:
print(st_df.describe())
print(hosp_df.describe())

       sub_region_2  ...  symptom:pancreatitis
count           0.0  ...                   0.0
mean            NaN  ...                   NaN
std             NaN  ...                   NaN
min             NaN  ...                   NaN
25%             NaN  ...                   NaN
50%             NaN  ...                   NaN
75%             NaN  ...                   NaN
max             NaN  ...                   NaN

[8 rows x 424 columns]
       cases_cumulative  ...  economic_support_index_for_display
count      4.424600e+04  ...                        48352.000000
mean       5.625675e+04  ...                           34.865673
std        3.481139e+05  ...                           33.538095
min        0.000000e+00  ...                            0.000000
25%        7.100000e+01  ...                            0.000000
50%        1.193000e+03  ...                           37.500000
75%        1.264000e+04  ...                           62.500000
max        7.277814e+06  ...     

## Preprocess the datasets

In [52]:
# Search trends dataset

#TODO: Preprocessing